# SDSS Galaxies vs quasars

We're now going to classify galaxy vs quasars in the Sloan Digital Sky Survey.

**Data** 

The dataset is at solutions/galaxyquasar.csv. I have extracted it myself from the SDSS database using the SQL query reported here

## Tasks

- Create arrays for the $(u - g)$, $(g - r)$, $(r - i)$, and $(i - z)$ colors. Also create an array with the class labels where galaxy = 0 and quasar = 1.
- Classify the dataset against the target label.
- Try some of the classification methods we've seen so far and evaluate the performance using the ROC curve.
- Remember to split the dataset into training and validation...

## Ideas

- Try using different colors (a subset of them first, than all together). Which is the most important feature?
- What are the colors that better satisfy or invalidate the "Naive" assumption of independence between the attributes?